## Imports

In [1]:
# Imports and set torch device
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt
import kaolin as kal
import cv2
from collections import defaultdict
import trimesh
from scipy.spatial.distance import cdist
import torch
import torch.nn.functional as F

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print('Torch will run on:', device)

object = 'bookshelf' 
obj_path = 'data/demo/' + object + '.obj'

Torch will run on: cuda:0


## Read Mesh

In [2]:
# Read mesh
mesh = kal.io.obj.import_mesh(
    obj_path,
    with_normals=True,
    with_materials=False,
)

vertices_tensor = mesh.vertices.to(device)
faces_tensor = mesh.faces.to(device)

vertices = vertices_tensor.detach().cpu().numpy()
faces = faces_tensor.detach().cpu().numpy()
colors =  mesh.vertex_normals.cpu().numpy()

print('Number of vertices: ', vertices.shape[0])
print('Number of faces: ', faces.shape[0])

Number of vertices:  5786
Number of faces:  8624


## Sample Points on mesh surface

In [3]:
# Visualize mesh
trimeshMesh = trimesh.Trimesh(vertices, faces)
# N = int(vertices.shape[0] * 2)
N = int(vertices.shape[0] / 2)
point_cloud, pt_to_face = trimesh.sample.sample_surface_even(trimeshMesh, N)
torchPC = torch.tensor(point_cloud, device=torch.device('cuda:0'), dtype=torch.float32)
face_to_all_pts = defaultdict(list)
for pt in range(len(point_cloud)):
    face_to_all_pts[pt_to_face[pt]].append(pt)
p = mp.plot(vertices, faces, colors, return_plot=True, shading={"wireframe": 0.1})
p.add_points(point_cloud, shading={"point_size": 0.1, "point_color": "green"})
plt.show()

only got 1877/2893 samples!


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.4272670…

c:\ProgramData\miniforge3\envs\test_proj\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


## Build SPC from Point Cloud

In [4]:
# Our SPC will contain a hierarchy of multiple levels
level = 8
torch.manual_seed(0)
# colors = torch.rand(point_cloud.shape[0], 3, device=device)
# colors = torch.tensor(point_cloud, device=device)
# colors = torch.tensor(point_cloud[:,:1]).to(device)
colors = torch.arange(0, point_cloud.shape[0]).unsqueeze(1).to(device)
# mp.plot(point_cloud, c = colors.cpu().numpy(), shading={'point_size':0.2})
# plt.show()
spc = kal.ops.conversions.pointcloud.unbatched_pointcloud_to_spc(
    pointcloud=torchPC, level=level, features=colors)

## Define functions for ray generation

In [22]:
def _normalized_grid(width, height, device='cuda'):
    """Returns grid[x,y] -> coordinates for a normalized window.
    
    Args:
        width, height (int): grid resolution
    """

    # These are normalized coordinates
    # i.e. equivalent to 2.0 * (fragCoord / iResolution.xy) - 1.0
    window_x = torch.linspace(-1, 1, steps=width, device=device) * (width / height)
    window_y = torch.linspace(1,- 1, steps=height, device=device)

    coord = torch.stack(torch.meshgrid(window_x, window_y)).permute(1,2,0)
    return coord


def look_at(camera_from, camera_to, width, height, mode='persp', fov=90.0, device='cuda'):
    """Vectorized look-at function, returns an array of ray origins and directions
    URL: https://www.scratchapixel.com/lessons/mathematics-physics-for-computer-graphics/lookat-function
    """

    camera_origin = torch.FloatTensor(camera_from).to(device)
    camera_view = F.normalize(torch.FloatTensor(camera_to).to(device) - camera_origin, dim=0)
    camera_right = F.normalize(torch.cross(camera_view, torch.FloatTensor([0,1,0]).to(device)), dim=0)
    camera_up = F.normalize(torch.cross(camera_right, camera_view), dim=0)

    coord = _normalized_grid(width, height, device=device)
    ray_origin = camera_right * coord[...,0,np.newaxis] * np.tan(np.radians(fov/2)) + \
                 camera_up * coord[...,1,np.newaxis] * np.tan(np.radians(fov/2)) + \
                 camera_origin + camera_view
    ray_origin = ray_origin.reshape(-1, 3)
    ray_offset = camera_view.unsqueeze(0).repeat(ray_origin.shape[0], 1)
    
    if mode == 'ortho': # Orthographic camera
        ray_dir = F.normalize(ray_offset, dim=-1)
    elif mode == 'persp': # Perspective camera
        ray_dir = F.normalize(ray_origin - camera_origin, dim=-1)
        ray_origin = camera_origin.repeat(ray_dir.shape[0], 1)
    else:
        raise ValueError('Invalid camera mode!')


    return ray_origin, ray_dir

## Generate Rays

In [125]:
# ray_o and ray_d ~ torch.Tensor (width x height, 3)
# represent rays origin and direction vectors
# camera_from=[-.5,2.5,-2.5]
camera_from = [0.,0.5,2.]
camera_to = [0,0.5,0]
ray_o, ray_d = look_at(
                        camera_from=camera_from,
                        camera_to=camera_to,
                        width=1024,
                        height=1024,
                        mode='persp',
                        fov=60,
                        device='cuda')
np.random.seed(42)
p = mp.plot(point_cloud, shading={'point_size':0.2, 'point_color':'black'}, return_plot=True)
p.add_points(ray_o[0].view(1,3).cpu().numpy(), shading={'point_size':0.2, 'point_color':'green'})
for i in range(10):
    p.add_lines(ray_o[0].view(1,3).cpu().numpy(), (ray_o[0] + 4.*ray_d[np.random.randint(ray_d.shape[0])]).view(1,3).cpu().numpy(), shading={'line_size':0.2, 'line_color':'blue'})
plt.show()
print(f'Total of {ray_o.shape[0]} rays generated.')

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.2782554…

Total of 1048576 rays generated.


## Manually define Ray

In [123]:
ray_o = torch.tensor([[0.,0.5,2.]], device=device)
ray_d = torch.tensor([[0,0.,-1]], device=device)
p = mp.plot(point_cloud, shading={'point_size':0.2, 'point_color':'black'}, return_plot=True)
p.add_lines(ray_o.cpu().numpy(), (ray_o + 2.*ray_d).cpu().numpy(), shading={'line_size':0.2, 'line_color':'blue'})
p.add_points(ray_o.cpu().numpy(), shading={'point_size':0.2, 'point_color':'green'})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.2782554…

## Ray Trace

In [126]:
octree, features = spc.octrees, spc.features
point_hierarchy, pyramid, prefix = spc.point_hierarchies, spc.pyramids[0], spc.exsum
nugs_ridx, nugs_pidx, depth = kal.render.spc.unbatched_raytrace(
    octree, point_hierarchy, pyramid, prefix, ray_o, ray_d, level)
masked_nugs = kal.render.spc.mark_pack_boundaries(nugs_ridx)
nugs_ridx = nugs_ridx[masked_nugs]
nugs_pidx = nugs_pidx[masked_nugs]
ridx = nugs_ridx.long()
pidx = nugs_pidx.long() - pyramid[1,level]
print(f'Ray Hits: {ridx.shape[0]}')

Ray Hits: 25268


## Show intersection with PC

In [127]:
p = mp.plot(point_cloud, shading={'point_size':0.2, 'point_color':'black'}, return_plot=True)
p.add_points(ray_o[0].view(1, -1).cpu().numpy(), shading={'point_size':0.5, 'point_color':'green'})
for i in range(5):
    id_p = np.random.randint(ridx.shape[0])
    p.add_lines(ray_o[0].view(1, -1).cpu().numpy(), 
                (ray_o[0] + 4*ray_d[ridx[id_p]]).view(1, -1).cpu().numpy(), 
                shading={'line_color':'blue'})
    m_hit = point_cloud[features[pidx[id_p]]]
    p.add_points(m_hit.reshape(1, -1), shading={'point_size':0.5, 'point_color':'blue'})
plt.show()

c:\ProgramData\miniforge3\envs\test_proj\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.2782554…